In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import requests
from bs4 import BeautifulSoup

In [2]:
# 1건의 대화 내용을 저장하는 클래스 => 질문과 답변이 한 쌍으로 저장되는 클래스
class Conversation:
    def __init__(self, question, answer):
        self.question = question
        self.answer = answer
    def __str__(self):
        return '질문: {}\n답변: {}'.format(self.question, self.answer)

In [4]:
c = Conversation('너... 누구야??', '글쎄... 알아서 뭐하시려고...')
print(c)

질문: 너... 누구야??
답변: 글쎄... 알아서 뭐하시려고...


In [7]:
# 75개의 대화 주제와 세부 대화 내용의 url을 크롤링해서 리턴시키는 함수
def getSubject():
    targetSite = 'https://basicenglishspeaking.com/daily-english-conversation-topics/'
    request = requests.get(targetSite)
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    
    subjects = []
    contentLink = [] # 세부 대화 내용의 url을 저장할 빈 리스트를 선언한다.
    divs = soup.findAll('div', {'class': 'tcb-col'})
    for div in divs:
        chapter = div.findAll('a')
        for a in chapter:
            subjects.append(a.text)
            # 세부 대화 내용의 url을 리스트에 추가한다.
            # print(a.get('href'))
            contentLink.append(a.get('href'))
    
    # 대화 주제와 대화 주제에 따른 세부 대화 내용의 url을 리턴시킨다.
    # 리턴되는 데이터가 2개가 아니라 subjects, contentLink를 1개의 튜플로 리턴시킨다.
    return subjects, contentLink

In [36]:
subjects, contentLink = getSubject()
# for i in range(len(subjects)):
    # print('{0:2d}. {1} - {2}'.format(i + 1, subjects[i], contentLink[i]))
    
# 대화 주제에 따른 모든 대화 내용을 저장할 빈 리스트를 선언한다.
conversation = [] # Conversation 클래스 객체를 저장한다.

i = 0 # 대화 주제의 개수를 카운트 하는 변수 => 인덱스
# 대화 주제의 개수만큼 반복하며 각각의 대화 주제에 다른 대화 내용을 읽는다.
for subject in subjects[:3]:
    conversation.append('{0:2d}. {1} - {2}'.format(i + 1, subjects[i], contentLink[i]))
    
    # 대화 주제별로 대화 내용을 크롤링할 페이지를 요청한다.
    request = requests.get(contentLink[i])
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # 대화 주제에 따른 대화 내용은 플레이 버튼(class 속성이 'sc_player_container1'인 div 태그)의 다음 형제이다.
    divs = soup.findAll('div', {'class': 'sc_player_container1'})
    
    # 각각의 대호 주제에 따른 문장의 개수만큼 반복한다.
    for div in divs:
        # index() 메소드로 전체 중에서 인수로 지정한 객체의 index 번호를 얻어올 수 있다.
        # divs.index(div) => 전체 버튼(divs)에서 특정 버튼(div)의 index 번호를 얻어온다.
        # print(divs.index(div))
        # index() 메소드 실행 결과가 짝수면 질문이고 홀수면 답변이다.
        
        # 크롤링할 데이터가 태그 내부에 작성된 것이 아니고 class 속성이 'sc_player_container1'인 div 태그의 다음 형제로
        # 작성되어 있다.
        # next_sibling => 다음 형제, prevoius_sibling => 이전 형제
        if divs.index(div) % 2 == 0: # 질문인가?
            # print('질문: {}'.format(div.next_sibling))
            question = div.next_sibling
        else:
            # print('답변: {}'.format(div.next_sibling))
            answer = div.next_sibling
            
            # Conversation 클래스 객체를 만드는 작업은 질문과 답변이 한 쌍이 되는 순간 실행한다.
            # 질문과 답변이 한 쌍이 되었으므로 Conversation 클래스 객체를 생성하고 질문과 답변을 저장한다.
            c = Conversation(question, answer)
            # 질문과 답변이 한 쌍으로 저장된 Conversation 클래스 객체를 대화 내용을 기억하는 conversation 리스트에 저장한다.
            conversation.append(c)
    
    # 대화 주제 1건이 처리 완료되면 다음 대화를 처리하기 위해서 i를 1증가시킨다.
    i += 1

In [37]:
for c in conversation:
    print(c)

 1. Family - https://basicenglishspeaking.com/family/
질문: How many people are there in your family?
답변: There are 5 people in my family: my father, mother, brother, sister, and me.
질문: Does your family live in a house or an apartment?
답변: We live in a house in the countryside.
질문: What does your father do?
답변: My father is a doctor. He works at the local hospital.
질문: How old is your mother?
답변: She is 40 years old, 1 year younger than my father.
질문: Do you have any siblings? What’s his/her name?
답변: Yes, I do. I have 1 elder brother, David, and 1 younger sister, Mary.
질문: Are you the oldest amongst your brothers and sisters?
답변: No, I’m not. I’m the second child in my family.
질문: What does your mother/father like?
답변: My father likes playing football and my mother likes cooking.
질문: Do your parents let you stay out late?
답변: Of course not. They always ask me to get home before 10 pm each night.
질문: Do you stay with your parents?
답변: Right now, no, but I used to.
질문: Does your family u